**This notebook is an exercise in the [Python](https://www.kaggle.com/learn/python) course.  You can reference the tutorial at [this link](https://www.kaggle.com/colinmorris/functions-and-getting-help).**

---


Functions are powerful. Try writing some yourself.

As before, don't forget to run the setup code below before jumping into question 1.

In [1]:
# SETUP. You don't need to worry for now about what this code does or how it works.
from learntools.core import binder; binder.bind(globals())
from learntools.python.ex2 import *
print('Setup complete.')

Setup complete.


# 1.

Complete the body of the following function according to its docstring.

HINT: Python has a built-in function `round`.

In [2]:
def round_to_two_places(num):
    """Return the given number rounded to two decimal places. 
    
    >>> round_to_two_places(3.14159)
    3.14
    """
    # Replace this body with your own code.
    # ("pass" is a keyword that does literally nothing. We used it as a placeholder
    # because after we begin a code block, Python requires at least one line of code)
    pass
    return round(num, 2)

# Check your answer
q1.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [3]:
# Uncomment the following for a hint
q1.hint()
# Or uncomment the following to peek at the solution
q1.solution()

<IPython.core.display.Javascript object>

<span style="color:#3366cc">Hint:</span> Run `help(round)` in the console (or in a code cell) to learn more about the round function. You'll need to use the function's optional second argument.

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python
return round(num, 2)
```

# 2.
The help for `round` says that `ndigits` (the second argument) may be negative.
What do you think will happen when it is? Try some examples in the following cell.

In [4]:
help(round)# Put your test code here
print(round(123,-2))
print(round(123,-3))
print(round(1.235,2))
print(round(1.2355,3))

Help on built-in function round in module builtins:

round(number, ndigits=None)
    Round a number to a given precision in decimal digits.
    
    The return value is an integer if ndigits is omitted or None.  Otherwise
    the return value has the same type as the number.  ndigits may be negative.

100
0
1.24
1.236


Can you think of a case where this would be useful?  Once you're ready, run the code cell below to see the answer and to receive credit for completing the problem.

In [5]:
# Check your answer (Run this code cell to receive credit!)
q2.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> As you've seen, `ndigits=-1` rounds to the nearest 10, `ndigits=-2` rounds to the nearest 100 and so on. Where might this be useful? Suppose we're dealing with large numbers:

> The area of Finland is 338,424 km²  
> The area of Greenland is 2,166,086 km²

We probably don't care whether it's really 338,424, or 338,425, or 338,177. All those digits of accuracy are just distracting. We can chop them off by calling `round()` with `ndigits=-3`:

> The area of Finland is 338,000 km²  
> The area of Greenland is 2,166,000 km²

(We'll talk about how we would get the commas later when we talk about string formatting :))


# 3.

In the previous exercise, the candy-sharing friends Alice, Bob and Carol tried to split candies evenly. For the sake of their friendship, any candies left over would be smashed. For example, if they collectively bring home 91 candies, they'll take 30 each and smash 1.

Below is a simple function that will calculate the number of candies to smash for *any* number of total candies.

Modify it so that it optionally takes a second argument representing the number of friends the candies are being split between. If no second argument is provided, it should assume 3 friends, as before.

Update the docstring to reflect this new behaviour.

In [6]:
def to_smash(total_candies,friends=3):
    """Return the number of leftover candies that must be smashed after distributing
    the given number of candies evenly between 3 friends.
    
    >>> to_smash(91)
    1
    """
    return total_candies % friends

# Check your answer
q3.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [7]:
q3.hint()

<IPython.core.display.Javascript object>

<span style="color:#3366cc">Hint:</span> Refer to the section of the last tutorial notebook where we talked about default arguments

In [8]:
q3.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python
def to_smash(total_candies, n_friends=3):
    return total_candies % n_friends
```

# 4. (Optional)

It may not be fun, but reading and understanding error messages will be an important part of your Python career.

Each code cell below contains some commented buggy code. For each cell...

1. Read the code and predict what you think will happen when it's run.
2. Then uncomment the code and run it to see what happens. (**Tip**: In the kernel editor, you can highlight several lines and press `ctrl`+`/` to toggle commenting.)
3. Fix the code (so that it accomplishes its intended purpose without throwing an exception)

<!-- TODO: should this be autochecked? Delta is probably pretty small. -->

In [9]:
def ruound_to_two_places(num):
    return round(num, 2)

ruound_to_two_places(9.9999)

10.0

In [10]:
 x = -10
 y = 5
 # Which of the two variables above has the smallest absolute value?
 smallest_abs = min(abs(x), abs(y))
help(min)
help(abs)
print(smallest_abs)

Help on built-in function min in module builtins:

min(...)
    min(iterable, *[, default=obj, key=func]) -> value
    min(arg1, arg2, *args, *[, key=func]) -> value
    
    With a single iterable argument, return its smallest item. The
    default keyword-only argument specifies an object to return if
    the provided iterable is empty.
    With two or more arguments, return the smallest argument.

Help on built-in function abs in module builtins:

abs(x, /)
    Return the absolute value of the argument.

5


In [11]:
 def f(x):
     y = abs(x)
     return y

 print(f(5))
print(f(-3))

5
3


# Keep Going

Nice job with the code. Next up, you'll learn about *conditionals*, which you'll need to **[write interesting programs](https://www.kaggle.com/colinmorris/booleans-and-conditionals)**. 

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/python/discussion) to chat with other learners.*